In [1]:
#from data_preprocess import FiducialDataProcess
import ipdb
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
import warnings

class NMF_Data(object):
	def __init__(self, dat_x, dat_y):
		self.dat_x = dat_x
		self.dat_y = dat_y
		self.nmf_features = []

	def create_nmf(self, test_size=500):
		x_train, x_test, y_train, y_test = train_test_split(self.dat_x, self.dat_y, random_state=1, test_size = test_size)
		print(x_train.shape, x_test.shape)
		
		nmf = NMF(n_components=100, random_state=0)
		nmf.fit(x_train)

		x_train_nmf = nmf.transform(x_train)
		x_test_nmf = nmf.transform(x_test)

		self.nmf_features.append(x_train_nmf)
		self.nmf_features.append(y_train)
		self.nmf_features.append(x_test_nmf)
		self.nmf_features.append(y_test)

	def get_nmf_features(self):
		return self.nmf_features

	def save_nmf(self, filename):
		self.create_nmf()
		np.save(filename, self.nmf_features)


if __name__ == '__main__':
    my_data = np.genfromtxt('dense_data_type_and_emot.csv', delimiter=',')
    np.random.seed(0)
    np.random.shuffle(my_data)	
    features_init = my_data[:,1:]
    features = features_init / features_init.max(axis=0)
    labels = my_data[:,0:2] 
    nmf_total = NMF_Data(features, labels)
    filename = 'nmf_features_type_emot.npy'
    nmf_total.save_nmf(filename)


	# dat = pd.read_csv("nmf_faducial.csv", index_col=0)
	# dat_x = dat.loc[:,'feature1':'feature6006']
	# dat_y = dat.loc[:, 'emotion_idx']
	# nmf_total = NMF_Data(dat_x, dat_y)
	# nmf_total.save_nmf()

/Users/hechongyu1995/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(2000, 3004) (500, 3004)


In [2]:
nmf_create = NMF_Data.create_nmf(nmf_total)
data_get = NMF_Data.get_nmf_features(nmf_total)

(2000, 3004) (500, 3004)


In [3]:
x_train, y_train, x_test, y_test = data_get[0], data_get[1], data_get[2], data_get[3]

### SVM for 0/1

In [4]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

warnings.filterwarnings('ignore')
svm_param_grid = {'C':[0.001,0.01,0.1,1,10,50,100,500],
                   'kernel':['rbf', 'linear', 'poly', 'sigmoid']}

svm_clf = SVC(**svm_param_grid)

grid = GridSearchCV(svm_clf,
                    param_grid=svm_param_grid,
                    cv=5,
                    verbose = 1,
                    scoring='accuracy')

grid.fit(x_train, y_train[:,0])
grid.best_params_, grid.best_score_

/Users/hechongyu1995/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/hechongyu1995/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:   41.9s finished


({'C': 500, 'kernel': 'linear'}, 0.9885)

In [5]:
svm_best = SVC(**grid.best_params_)
svm_best.fit(x_train,y_train[:,0])
score_svm = svm_best.score(x_test,y_test[:,0])
score_svm

0.988

In [6]:
print ('overall accuracy is ' + str(score_svm))

overall accuracy is 0.988


### SVM for multi emotion classification

In [7]:
svm_param_grid = {'C':[0.001,0.01,0.1,1,10,50,100,500],
                   'kernel':['rbf', 'linear', 'poly', 'sigmoid'],
                   'decision_function_shape': ['ovr', 'ovo']}

svm_clf_multi = SVC(**svm_param_grid)

grid = GridSearchCV(svm_clf_multi,
                    param_grid=svm_param_grid,
                    cv=5,
                    verbose = 1,
                    scoring='accuracy')

grid.fit(x_train, y_train[:,1])
grid.best_params_, grid.best_score_

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:  3.1min finished


({'C': 100, 'decision_function_shape': 'ovr', 'kernel': 'linear'}, 0.4825)

In [8]:
svm_best = SVC(**grid.best_params_)
svm_best.fit(x_train,y_train[:,1])
score_svm = svm_best.score(x_test,y_test[:,1])
score_svm

0.474

In [9]:
y_train_simple, y_test_simple = y_train[:,0], y_test[:,0]
x_train_0 = x_train[np.where(y_train_simple==0)]
y_train_compund_0 = y_train[:,1][np.where(y_train_simple==0)]

svm_param_grid = {'C':[0.001,0.01,0.1,1,10,50,100,500],
                   'kernel':['rbf', 'linear', 'poly', 'sigmoid'],
                   'decision_function_shape': ['ovr', 'ovo']}

svm_clf_compund_0 = SVC(**svm_param_grid)

grid = GridSearchCV(svm_clf_compund_0,
                    param_grid=svm_param_grid,
                    cv=5,
                    verbose = 1,
                    scoring='accuracy')

grid.fit(x_train_0, y_train_compund_0)
grid.best_params_, grid.best_score_

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:   16.5s finished


({'C': 500, 'decision_function_shape': 'ovr', 'kernel': 'rbf'},
 0.7370078740157481)

In [10]:
x_test_0 = x_test[np.where(y_test_simple==0)]
y_test_compund_0 = y_test[:,1][np.where(y_test_simple==0)]

svm_best_compound_0 = SVC(**grid.best_params_)
svm_best_compound_0.fit(x_train_0, y_train_compund_0)
score_svm_compound_0 = svm_best_compound_0.score(x_test_0, y_test_compund_0)
score_svm_compound_0

0.7543859649122807

In [11]:
y_train_simple, y_test_simple = y_train[:,0], y_test[:,0]
x_train_1 = x_train[np.where(y_train_simple==1)]
y_train_compund_1 = y_train[:,1][np.where(y_train_simple==1)]

svm_param_grid = {'C':[0.001,0.01,0.1,1,10,50,100,500],
                   'kernel':['rbf', 'linear', 'poly', 'sigmoid'],
                   'decision_function_shape': ['ovr', 'ovo']}

svm_clf_compund_1 = SVC(**svm_param_grid)

grid = GridSearchCV(svm_clf_compund_1,
                    param_grid=svm_param_grid,
                    cv=5,
                    verbose = 1,
                    scoring='accuracy')

grid.fit(x_train_1, y_train_compund_1)
grid.best_params_, grid.best_score_

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:  1.4min finished


({'C': 50, 'decision_function_shape': 'ovr', 'kernel': 'linear'},
 0.49157509157509155)

In [12]:
x_test_1 = x_test[np.where(y_test_simple==1)]
y_test_compund_1 = y_test[:,1][np.where(y_test_simple==1)]

svm_best_compound_1 = SVC(**grid.best_params_)
svm_best_compound_1.fit(x_train_1, y_train_compund_1)
score_svm_compound_1 = svm_best_compound_1.score(x_test_1, y_test_compund_1)
score_svm_compound_1

0.46200607902735563

In [13]:
overall_acc = (len(y_test_compund_0)*score_svm_compound_0+len(y_test_compund_1)*score_svm_compound_1)/len(y_test)
print ('overall SVM accuracy is ' + str(overall_acc))

overall SVM accuracy is 0.562


### RF for multi emotion classification

In [14]:
from sklearn.ensemble import RandomForestClassifier
y_train_simple, y_test_simple = y_train[:,0], y_test[:,0]
x_train_0 = x_train[np.where(y_train_simple==0)]
y_train_compund_0 = y_train[:,1][np.where(y_train_simple==0)]

rf_param_grid = {'n_estimators':[20,50,100,200],
                 'max_depth':[2,5,10,15],
                 'min_samples_split':np.arange(2,10,2),
                 'max_features':np.arange(5,25,5),
                 }

rf_clf_compund_0 = RandomForestClassifier(**rf_param_grid)

rf_grid = GridSearchCV(rf_clf_compund_0,
                    param_grid=rf_param_grid,
                    cv=5,
                    verbose=1,
                    scoring='accuracy')

rf_grid.fit(x_train_0, y_train_compund_0)
rf_grid.best_params_, rf_grid.best_score_

Fitting 5 folds for each of 256 candidates, totalling 1280 fits


[Parallel(n_jobs=1)]: Done 1280 out of 1280 | elapsed:  5.0min finished


({'max_depth': 10,
  'max_features': 20,
  'min_samples_split': 4,
  'n_estimators': 100},
 0.8582677165354331)

In [15]:
x_test_0 = x_test[np.where(y_test_simple==0)]
y_test_compund_0 = y_test[:,1][np.where(y_test_simple==0)]

rf_best_compound_0 = RandomForestClassifier(**rf_grid.best_params_)
rf_best_compound_0.fit(x_train_0, y_train_compund_0)
score_rf_compound_0 = rf_best_compound_0.score(x_test_0, y_test_compund_0)
score_rf_compound_0

0.8538011695906432

In [16]:
y_train_simple, y_test_simple = y_train[:,0], y_test[:,0]
x_train_1 = x_train[np.where(y_train_simple==1)]
y_train_compund_1 = y_train[:,1][np.where(y_train_simple==1)]

rf_param_grid_1 = {'n_estimators':[20,50,100,200],
                 'max_depth':[2,5,10,15],
                 'min_samples_split':np.arange(2,10,2),
                 'max_features':np.arange(5,25,5),
                 }

rf_clf_compund_1 = RandomForestClassifier(**rf_param_grid_1)

rf_grid_1 = GridSearchCV(rf_clf_compund_1,
                    param_grid=rf_param_grid_1,
                    cv=5,
                    verbose=1,
                    scoring='accuracy')

rf_grid_1.fit(x_train_1, y_train_compund_1)
rf_grid_1.best_params_, rf_grid_1.best_score_

Fitting 5 folds for each of 256 candidates, totalling 1280 fits


[Parallel(n_jobs=1)]: Done 1280 out of 1280 | elapsed: 11.3min finished


({'max_depth': 15,
  'max_features': 20,
  'min_samples_split': 2,
  'n_estimators': 200},
 0.6043956043956044)

In [17]:
x_test_1 = x_test[np.where(y_test_simple==1)]
y_test_compund_1 = y_test[:,1][np.where(y_test_simple==1)]

rf_best_compound_1 = RandomForestClassifier(**rf_grid_1.best_params_)
rf_best_compound_1.fit(x_train_1, y_train_compund_1)
score_rf_compound_1 = rf_best_compound_1.score(x_test_1, y_test_compund_1)
score_rf_compound_1

0.5775075987841946

In [18]:
overall_rf_acc = (len(y_test_compund_0)*score_rf_compound_0+len(y_test_compund_1)*score_rf_compound_1)/len(y_test)
print ('overall RF accuracy is ' + str(overall_rf_acc))

overall RF accuracy is 0.672


### GBDT for multi emotion classification

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
y_train_simple, y_test_simple = y_train[:,0], y_test[:,0]
x_train_0 = x_train[np.where(y_train_simple==0)]
y_train_compund_0 = y_train[:,1][np.where(y_train_simple==0)]

gbdt_param_grid = {'learning_rate':[0.001,0.01,0.1],
                   'n_estimators':[20,50,100,200,500],
                   'max_depth':[5,10,15,20],
                   'max_features':np.arange(5,20,5),
                   }

gbdt_clf_compund_0 = GradientBoostingClassifier(**gbdt_param_grid)

gbdt_grid = GridSearchCV(gbdt_clf_compund_0,
                    param_grid=gbdt_param_grid,
                    cv=5,
                    verbose=1,
                    scoring='accuracy')

gbdt_grid.fit(x_train_0, y_train_compund_0)
gbdt_grid.best_params_, gbdt_grid.best_score_

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=1)]: Done 900 out of 900 | elapsed: 32.2min finished


({'learning_rate': 0.001,
  'max_depth': 15,
  'max_features': 10,
  'n_estimators': 500},
 0.8472440944881889)

In [27]:
x_test_0 = x_test[np.where(y_test_simple==0)]
y_test_compund_0 = y_test[:,1][np.where(y_test_simple==0)]

gbdt_best_compound_0 = GradientBoostingClassifier(**gbdt_grid.best_params_)
gbdt_best_compound_0.fit(x_train_0, y_train_compund_0)
score_gbdt_compound_0 = gbdt_best_compound_0.score(x_test_0, y_test_compund_0)
score_gbdt_compound_0

0.8187134502923976

In [30]:
y_train_simple, y_test_simple = y_train[:,0], y_test[:,0]
x_train_1 = x_train[np.where(y_train_simple==1)]
y_train_compund_1 = y_train[:,1][np.where(y_train_simple==1)]

gbdt_param_grid_1 = {'learning_rate':[0.001,0.01,0.1],
                   'n_estimators':[20,50,100,200],
                   'max_depth':[5,10,15,20],
                   'max_features':np.arange(5,20,5),
                   }

gbdt_clf_compund_1 = GradientBoostingClassifier(**gbdt_param_grid_1)

gbdt_grid_1 = GridSearchCV(gbdt_clf_compund_1,
                    param_grid=gbdt_param_grid_1,
                    cv=5,
                    verbose=1,
                    scoring='accuracy')

gbdt_grid_1.fit(x_train_1, y_train_compund_1)
gbdt_grid_1.best_params_, gbdt_grid_1.best_score_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed: 62.4min finished


({'learning_rate': 0.1,
  'max_depth': 5,
  'max_features': 15,
  'n_estimators': 200},
 0.5802197802197803)

In [48]:
x_test_1 = x_test[np.where(y_test_simple==1)]
y_test_compund_1 = y_test[:,1][np.where(y_test_simple==1)]

gbdt_best_compound_1 = GradientBoostingClassifier(**gbdt_grid_1.best_params_)
gbdt_best_compound_1.fit(x_train_1, y_train_compund_1)
score_gbdt_compound_1 = gbdt_best_compound_1.score(x_test_1, y_test_compund_1)
score_gbdt_compound_1

0.5562310030395137

In [49]:
overall_rf_acc = (len(y_test_compund_0)*score_gbdt_compound_0+len(y_test_compund_1)*score_gbdt_compound_1)/len(y_test)
print ('overall GBDT accuracy is ' + str(overall_rf_acc))

overall GBDT accuracy is 0.646


### Votingclassifier for multi emotion classification

In [55]:
from sklearn.ensemble import VotingClassifier

estimators=[('rf',rf_best_compound_0),('gbdt',gbdt_best_compound_0),('rf2',rf_best_compound_0)]
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(x_train_0, y_train_compund_0)
score0 = ensemble.score(x_test_0, y_test_compund_0)
score0

0.8538011695906432

In [50]:
estimators=[('rf',rf_best_compound_0),('gbdt',gbdt_best_compound_0),('rf2',rf_best_compound_0)]
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(x_train_1, y_train_compund_1)
score1 = ensemble.score(x_test_1, y_test_compund_1)
score1

0.60790273556231

In [56]:
overall_rf_acc = (len(y_test_compund_0)*score0+len(y_test_compund_1)*score1)/len(y_test)
print ('overall votingclassifier accuracy is ' + str(overall_rf_acc))

overall votingclassifier accuracy is 0.692
